In [ ]:
# Based on the GP algorithm to judge the seizure state of the patient
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# 总的神经元数量
Num=1000
# 输入的最大幅值
AmpStim=80 # 80,92
# 输入的持续时间
plat = 1000
# 输入的噪声强度
TauP=100
# 静息时的输入强度
rest=6
# # AdEx突触的传递时间
# tau_S_AdEx=6
# # AdEx耐火时间，不应期
# T_ref_AdEx=40
# # AdEx突触的传递时间
# tau_S_HH=2
# # HH耐火时间，不应期
# T_ref_HH=40

ms=1e-3
mV=1e-3
mA=1e-3
nA=1e-9
pA=1e-12
nS=1e-9
pF=1e-12

# whole world time
tick_time=0
# minium gap time3
dt=0.1*ms
# total time
TotTime=4000*ms
# time of fire
fire1_num=0
fire2_num=0
time_sim=np.arange(0, TotTime, dt)
# average data analysis
bin_num=1000
N1=int(Num*0.2)
N2=int(Num*0.8)
N1_A=int(N1*0.75)
N1_H=int(N1*0.25)
N2_A=int(N2*0.75)
N2_H=int(N2*0.25)
N_in=int(N2)

In [ ]:
# load the membrane potential of neurons
output_path = "C:\Parallel_World\BaiduSyncdisk\My_World\Software_World\BBQ_17\Seizure_Ref\My_Network\My_Union_Network\My_Union_Data\All_rest_6_AmpStim_80_TauP_100_tau_S_1_3_T_ref_10_10_Num_1000A.npz"
loaded_data=np.load(output_path)
neuron_potential=loaded_data['neuron_potential']
neuron_name=loaded_data['neuron_name']
fire_matrix1=loaded_data['fire_matrix1']
fire_matrix2=loaded_data['fire_matrix2']
neuron_potential_after=loaded_data['neuron_potential_after']

average_potential_A1=np.mean(neuron_potential[0:N1_A,:],axis=0)
average_potential_A2=np.mean(neuron_potential[N1_A:N1_A+N2_A,:],axis=0)
average_potential_H1=np.mean(neuron_potential[N1_A+N2_A:N1_A+N2_A+N1_H,:],axis=0)
average_potential_H2=np.mean(neuron_potential[N1_A+N2_A+N1_H:N1_A+N2_A+N1_H+N2_H,:],axis=0)
average_potential=np.mean(neuron_potential[0:Num,:],axis=0)

average_potential_after_A1=np.mean(neuron_potential_after[0:N1_A,:],axis=0)
average_potential_after_A2=np.mean(neuron_potential_after[N1_A:N1_A+N2_A,:],axis=0)
average_potential_after_H1=np.mean(neuron_potential_after[N1_A+N2_A:N1_A+N2_A+N1_H,:],axis=0)
average_potential_after_H2=np.mean(neuron_potential_after[N1_A+N2_A+N1_H:N1_A+N2_A+N1_H+N2_H,:],axis=0)
average_potential_after=np.mean(neuron_potential_after[0:Num,:],axis=0)

In [ ]:
fig1=plt.figure(figsize=(60,65))
ax1=fig1.add_subplot(511)
ax2=fig1.add_subplot(512)
ax3=fig1.add_subplot(513)
ax4=fig1.add_subplot(514)
ax5=fig1.add_subplot(515)
ax1.plot(time_sim,average_potential_A1)
ax2.plot(time_sim,average_potential_A2)
ax3.plot(time_sim,average_potential_H1)
ax4.plot(time_sim,average_potential_H2)
ax5.plot(time_sim,average_potential)

In [ ]:
fig2=plt.figure(figsize=(60,65))
bx1=fig2.add_subplot(511)
bx2=fig2.add_subplot(512)
bx3=fig2.add_subplot(513)
bx4=fig2.add_subplot(514)
bx5=fig2.add_subplot(515)
bx1.plot(time_sim,average_potential_after_A1)
bx2.plot(time_sim,average_potential_after_A2)
bx3.plot(time_sim,average_potential_after_H1)
bx4.plot(time_sim,average_potential_after_H2)
bx5.plot(time_sim,average_potential_after)

In [ ]:
# 现在开始计算其对应的相关性
# 首先是需要构建对应的矩阵
# 计算GP算法的时间延迟参数（自相关法）
N=1000
def get_tau(imf):
    if (len(imf) != N):
        print('请输入指定的数据长度！')  # N为指定数据长度
        return 0
    elif (isinstance(imf, np.ndarray) != True):
        print('数据格式错误！')
        return 0
    else:
        j = 1  # j为固定值
        tau_max = 1000
        Rall = np.zeros(tau_max)
        for tau in range(tau_max):
            R = 0
            for i in range(N - j * tau):
                R += imf[i] * imf[i + j * tau]
            Rall[tau] = R / (N - j * tau)
        for tau in range(tau_max):
            if Rall[tau] < (Rall[0] * 0.6321):
                break
        return tau
    
def GP(imf,tau):            #GP算法求关联维数
    if (len(imf) != N):
        print('请输入指定的数据长度！')   # N为指定数据长度
        return
    elif (isinstance(imf, np.ndarray) != True):
        print('数据格式错误！')
        return
    else:
        m_max=10                  #最大嵌入维数
        ss=50                     #r的步长
        fig=plt.figure()
        print(m_max)
        for m in range(1,m_max+1):
            print('m=',m)
            i_num = N - (m - 1) * tau
            kj_m = np.zeros((i_num, m))  # m维重构相空间
            for i in range(i_num):
                for j in range(m):
                    kj_m[i][j] = imf[i + j * tau]
            dist_min, dist_max = np.linalg.norm(kj_m[0] - kj_m[1]), np.linalg.norm(kj_m[0] - kj_m[1])
            Dist_m = np.zeros((i_num, i_num))  # 两向量之间的距离
            for i in range(i_num):
                for k in range(i_num):
                    D= np.linalg.norm(kj_m[i] - kj_m[k])
                    if(D>dist_max):
                        dist_max=D
                    elif(D>0 and D<dist_min):
                        dist_min=D
                    Dist_m[i][k] = D
            dr=(dist_max-dist_min)/(ss-1)           #r的间距
            r_m=[]
            Cr_m=[]
            for r_index in range(ss):
                r=dist_min+r_index*dr
                r_m.append(r)
                Temp=np.heaviside(r-Dist_m,1)
                for i in range(i_num):
                    Temp[i][i]=0
                Cr_m.append(np.sum(Temp))
            r_m=np.log(np.array((r_m)))
            Cr_m=np.log(np.array((Cr_m))/(i_num*(i_num-1)))
            plt.plot(r_m,Cr_m)
        plt.show()


In [ ]:
rest_potential=average_potential[10000:11000]
tau=get_tau(rest_potential)
print(tau)
GP(rest_potential,tau)

In [ ]:
import numpy as np
from scipy.fftpack import fft
from scipy import fftpack
import matplotlib.pyplot as plt

N_ft=2000         #时频域的点数

# 计算GP算法的时间延迟参数（自相关法）
def get_tau(imf):
    if (len(imf) != N_ft):
        print('请输入指定的数据长度！')  # 需要更改，比如弹出对话框
        return 0,0,0
    elif (isinstance(imf, np.ndarray) != True):
        print('数据格式错误！')
        return 0,0,0
    else:
        j = 1  # j为固定值
        tau_max = 200
        Rall = np.zeros(tau_max)
        for tau in range(tau_max):
            R = 0
            for i in range(N_ft - j * tau):
                R += imf[i] * imf[i + j * tau]
            Rall[tau] = R / (N_ft - j * tau)
        for tau in range(tau_max):
            if Rall[tau] < (Rall[0] * 0.6321):
                break
            tauall=np.arange(tau_max)
        return tauall,Rall,tau


# 计算GP算法的嵌入维数(假近邻算法)
def get_m(imf, tau):
    if (len(imf) != N_ft):
        print('请输入指定的数据长度！')  # 需要更改，比如弹出对话框
        return 0, 0
    elif (isinstance(imf, np.ndarray) != True):
        print('数据格式错误！')
        return 0, 0
    else:
        m_max = 10
        P_m_all = []  # m_max-1个假近邻点百分率
        for m in range(1, m_max + 1):
            i_num = N_ft - (m - 1) * tau
            kj_m = np.zeros((i_num, m))  # m维重构相空间
            for i in range(i_num):
                for j in range(m):
                    kj_m[i][j] = imf[i + j * tau]
            if (m > 1):
                index = np.argsort(Dist_m)
                a_m = 0  # 最近邻点数
                for i in range(i_num):
                    temp = 0
                    for h in range(i_num):
                        temp = index[i][h]
                        if (Dist_m[i][temp] > 0):
                            break
                    D = np.linalg.norm(kj_m[i] - kj_m[temp])
                    D = np.sqrt((D * D) / (Dist_m[i][temp] * Dist_m[i][temp]) - 1)
                    if (D > 10):
                        a_m += 1
                P_m_all.append(a_m / i_num)
            i_num_m = i_num - tau
            Dist_m = np.zeros((i_num_m, i_num_m))  # 两向量之间的距离
            for i in range(i_num_m):
                for k in range(i_num_m):
                    Dist_m[i][k] = np.linalg.norm(kj_m[i] - kj_m[k])
        P_m_all = np.array(P_m_all)
        m_all = np.arange(1, m_max)
        return m_all, P_m_all


# GP算法求关联维数(时频域特征)
def GP(imf, tau):
    if (len(imf) != N_ft):
        print('请输入指定的数据长度！')  # 需要更改，比如弹出对话框
        return
    elif (isinstance(imf, np.ndarray) != True):
        print('数据格式错误！')
        return
    else:
        m_max = 10  # 最大嵌入维数
        ss = 50  # r的步长
        fig = plt.figure(1)
        for m in range(1, m_max + 1):
            i_num = N_ft - (m - 1) * tau
            kj_m = np.zeros((i_num, m))  # m维重构相空间
            for i in range(i_num):
                for j in range(m):
                    kj_m[i][j] = imf[i + j * tau]
            dist_min, dist_max = np.linalg.norm(kj_m[0] - kj_m[1]), np.linalg.norm(kj_m[0] - kj_m[1])
            Dist_m = np.zeros((i_num, i_num))  # 两向量之间的距离
            for i in range(i_num):
                for k in range(i_num):
                    D = np.linalg.norm(kj_m[i] - kj_m[k])
                    if (D > dist_max):
                        dist_max = D
                    elif (D > 0 and D < dist_min):
                        dist_min = D
                    Dist_m[i][k] = D
            dr = (dist_max - dist_min) / (ss - 1)  # r的间距
            r_m = []
            Cr_m = []
            for r_index in range(ss):
                r = dist_min + r_index * dr
                r_m.append(r)
                Temp = np.heaviside(r - Dist_m, 1)
                for i in range(i_num):
                    Temp[i][i] = 0
                Cr_m.append(np.sum(Temp))
            r_m = np.log(np.array((r_m)))
            Cr_m = np.log(np.array((Cr_m)) / (i_num * (i_num - 1)))
            plt.plot(r_m, Cr_m)
        plt.xlabel('ln(r)')
        plt.ylabel('ln(C)')
        plt.show()

if __name__=='__main__':
    # 检验关联维数程序
    t = []
    f1 = 25
    f2 = 30
    for i in range(N_ft):
        t.append(i * 0.001)
    t = np.array(t)
    # yu = np.ones(M * N)
    AEall = np.sin(t * 2 * np.pi * f1) + np.sin(t * 2 * np.pi * f2)  #在这里直接改信号


    tauall, Rall, tau = get_tau(AEall)
    print(tau)
    m, P = get_m(AEall, tau)
    GP(AEall, 1)
    fig2 = plt.figure(2)
    yu = np.ones(len(tauall)) * Rall[0] * 0.6321
    plt.plot(tauall, Rall)
    plt.plot(tauall, yu)
    plt.xlabel('tau')
    plt.ylabel('R')
    plt.show()
    fig3 = plt.figure(3)
    plt.plot(m, P)
    plt.xlabel('m')
    plt.ylabel('P')
    plt.show()
